In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# data columns

# videoId,
# title,
# description,
# tags,
# viewCount,
# likeCount,
# favoriteCount,
# commentCount,
# publishedAt,
# channelId,
# channelTitle,
# country

use_columns = ['videoId', 'title', 'description', 'tags', 'viewCount', 'likeCount', 'favoriteCount', 'commentCount', 'publishedAt', 'channelId', 'channelTitle', 'country']


In [ ]:
import json
import pandas as pd
import numpy as np
from urllib import parse
import requests
from youtubeapi import YoutubeApi
from utils import *
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import seaborn as sns

email = 'shockzinfinity@gmail.com'
url = 'https://youtube.googleapis.com/youtube/v3/'

api = YoutubeApi(apiKey, use_columns)

searchQuery = '코이티비 베트남'
# maxResults = 15

In [ ]:
datas = api.search_videos(searchQuery)
df = api.get_video_dataframe(datas, '베트남')
df

In [ ]:
df.info()

In [ ]:
df.to_csv('./data/vietnam_youtube.csv', encoding='utf-8', index=False)

In [ ]:
asia_data = pd.read_csv('./data/asia_data_kor.csv')
asia_data['Year'].fillna(method='ffill', inplace=True)
asia_data

In [ ]:
year = df.loc[0, 'publishedAtYear']
month = df.loc[0, 'publishedAtMonth']

display(year, month)

In [ ]:
# 해당 동영상 년/월 의 방문자 수
masking = (asia_data['Year'] == (str(year) + '년')) & (asia_data['month'] == (str(month) + '월'))
published_asia_data = asia_data[masking]
display(published_asia_data)

# 해당 동영상 1년 전/후 의 방문자 수
prev_masking = (asia_data['Year'] == (str(year - 1) + '년')) & (asia_data['month'] == (str(month) + '월'))
prev_asia_data = asia_data[prev_masking]
display(prev_asia_data)

next_masking = (asia_data['Year'] == (str(year + 1) + '년')) & (asia_data['month'] == (str(month) + '월'))
next_asia_data = asia_data[next_masking]
display(next_asia_data)


In [ ]:
idx_pub = published_asia_data.index.values[0]

prev_idx = prev_asia_data.index.values[0]
next_idx = next_asia_data.index.values[0]

# prev_period = asia_data.iloc[prev_idx:idx_pub+1, :]
prev_period = asia_data.iloc[prev_idx:idx_pub+1, asia_data.columns.get_loc('베트남_명수')]
display(prev_period)
# next_period = asia_data.iloc[idx_pub:next_idx+1, :]
next_period = asia_data.iloc[idx_pub:next_idx+1, asia_data.columns.get_loc('베트남_명수')]
display(next_period)


In [ ]:
sns.lineplot(data=prev_period)